# core

> This module contains all the core functions used in the library.

In [ ]:
#| default_exp core

In [ ]:
#| export
import logging
import os

from datasets import concatenate_datasets, Dataset
from rich.logging import RichHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(RichHandler(rich_tracebacks=True))
# Turn off logging for datasets
logging.getLogger("datasets").setLevel(logging.ERROR)

/home/nathan/miniconda3/envs/squeakily/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from datasets import load_dataset
from squeakily.filter import check_char_repetition, check_flagged_words, minhash_dedup
from squeakily.clean import remove_empty_lines, normalize_whitespace

In [ ]:
#| export
class Pipeline:
    """
    A pipeline is a collection of datasources and their associated transformations to be run.
    """
    def __init__(
        self,
        datasources # The datasources to be run
    ):
        self.datasources = datasources
    
    def run(
        self,
        global_filters=[], # Filters to be run at the dataset level rather than the example level
        global_cleaners=[], # Cleaners to be run at the dataset level rather than the example level
        cleaning_first=False, # Whether to run the cleaning transformations first
        globals_first=False, # Whether to run the global transformations first
    ):
        """
        Run the pipeline.
        """
        for datasource in self.datasources:
            dataset = datasource["dataset"]
            column = datasource["columns"][0]
            logger.info(f"Running datasource: {dataset.builder_name}")
            if cleaning_first:
                for c in datasource["cleaners"]:
                    logger.info(f"Running cleaner: {c.__name__} on {column}")
                    dataset = dataset.map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )
                for f in datasource["filters"]:
                    logger.info(f"Running filter: {f.__name__} on {column}")
                    dataset = dataset.filter(
                        lambda x: f(x[column]),
                        num_proc=os.cpu_count(),
                    )
            else:
                for f in datasource["filters"]:
                    logger.info(f"Running filter: {f.__name__} on {column}")
                    dataset = dataset.filter(
                        lambda x: f(x[column]),
                        num_proc=os.cpu_count(),
                    )
                for c in datasource["cleaners"]:
                    logger.info(f"Running cleaner: {c.__name__} on {column}")
                    dataset = dataset.map(
                        lambda x: {column: c(x[column])},
                        num_proc=os.cpu_count(),
                    )
        
        if len(global_filters) > 0:
            # concatenate all datasets
            datasets = [
                d["dataset"] for d in self.datasources
                if not d.get("skip_global", False)
            ]
            global_column = self.datasources[0]["columns"][0]
            global_dataset = concatenate_datasets(datasets)

            # Add a column representing the original dataset name
            md = []
            for d in datasets:
                md.extend([d.builder_name] * len(d))
            meta_data = Dataset.from_dict({"meta_data": md})
            global_dataset_with_meta = concatenate_datasets([global_dataset, meta_data], axis=1)

            # Run the global filters
            for f in global_filters:
                logger.info(f"Running global filter: {f.__name__}")
                global_dataset_with_meta = f(global_dataset_with_meta, global_column)

            # Split the dataset back up
            for i, dataset in enumerate(datasets):
                self.datasources[i]["dataset"] = global_dataset_with_meta.filter(
                    lambda x: x["meta_data"] == dataset.builder_name,
                    num_proc=os.cpu_count(),
                )

In [ ]:
#|echo: true
show_doc(Pipeline.run)

---

[source](https://github.com/CarperAI/squeakily/blob/main/squeakily/core.py#L30){target="_blank" style="float:right; font-size:smaller"}

### Pipeline.run

>      Pipeline.run (global_filters=[], global_cleaners=[],
>                    cleaning_first=False, globals_first=False)

Run the pipeline.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| global_filters | list | [] | Filters to be run at the dataset level rather than the example level |
| global_cleaners | list | [] | Cleaners to be run at the dataset level rather than the example level |
| cleaning_first | bool | False | Whether to run the cleaning transformations first |
| globals_first | bool | False | Whether to run the global transformations first |

In [ ]:
ds = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")
logger.info(f"Original dataset size: {len(ds)}")
datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
    },
    # ...
]

global_filters = [minhash_dedup]
pipeline = Pipeline(datasources)
pipeline.run(global_filters=global_filters)
logger.info(f"Final dataset size: {len(pipeline.datasources[0]['dataset'])}")

[11/26/22 16:54:24] INFO     Original dataset size: 18014                                           ]8;id=117692;file:///tmp/ipykernel_132178/1617443191.py\1617443191.py]8;;\:]8;id=350765;file:///tmp/ipykernel_132178/1617443191.py#2\2]8;;\

                    INFO     Running datasource: wikitext                                          ]8;id=961561;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=671094;file:///tmp/ipykernel_132178/2234059382.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=573567;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=577814;file:///tmp/ipykernel_132178/2234059382.py#41\41]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=910600;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=856234;file:///tmp/ipykernel_132178/2234059382.py#41\41]8;;\

                    INFO     Running cleaner: remove_empty_lines on text                           ]8;id=615791;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=255930;file:///tmp/ipykernel_132178/2234059382.py#47\47]8;;\

...

                    INFO     Dataset columns: ['__id__', '__signature__']                             ]8;id=915580;file:///work/squeakily/squeakily/filter.py\filter.py]8;;\:]8;id=790300;file:///work/squeakily/squeakily/filter.py#330\330]8;;\

Indexing signatures...: 100%|██████████| 18014/18014 [00:01<00:00, 16042.04it/s]


[11/26/22 16:54:52] INFO     Dataset columns: ['__id__', '__neighbors__']                             ]8;id=777968;file:///work/squeakily/squeakily/filter.py\filter.py]8;;\:]8;id=600468;file:///work/squeakily/squeakily/filter.py#364\364]8;;\

Iterating over components...: 100%|██████████| 10560/10560 [00:00<00:00, 2971610.21it/s]


[11/26/22 16:55:04] INFO     Final dataset size: 10560                                             ]8;id=574671;file:///tmp/ipykernel_132178/1617443191.py\1617443191.py]8;;\:]8;id=742687;file:///tmp/ipykernel_132178/1617443191.py#16\16]8;;\

In [ ]:
# test the ability to skip global filters

ds_1 = load_dataset("wikitext", "wikitext-103-v1", split="train[:1%]")


datasources = [
    {
        "dataset": ds,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
        "skip_global": False,
    },
    {
        "dataset": ds_1,
        "columns": ["text"],
        "filters": [check_char_repetition, check_flagged_words],
        "cleaners": [remove_empty_lines, normalize_whitespace],
        "skip_global": True,
    },
    # ...
]
pipeline = Pipeline(datasources)
pipeline.run(global_filters=global_filters)
logger.info(f"Final dataset size: {len(pipeline.datasources[0]['dataset'])}")

[11/26/22 17:07:44] INFO     Running datasource: wikitext                                          ]8;id=750857;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=188292;file:///tmp/ipykernel_132178/2234059382.py#25\25]8;;\

                    INFO     Running filter: check_char_repetition on text                         ]8;id=745952;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=311574;file:///tmp/ipykernel_132178/2234059382.py#41\41]8;;\

                    INFO     Running filter: check_flagged_words on text                           ]8;id=891500;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=795588;file:///tmp/ipykernel_132178/2234059382.py#41\41]8;;\

                    INFO     Running cleaner: remove_empty_lines on text                           ]8;id=253503;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=384611;file:///tmp/ipykernel_132178/2234059382.py#47\47]8;;\

                    INFO     Running cleaner: normalize_whitespace on text                         ]8;id=7261;file:///tmp/ipykernel_132178/2234059382.py\2234059382.py]8;;\:]8;id=814530;file:///tmp/ipykernel_132178/2234059382.py#47\47]8;;\

...

                    INFO     Dataset columns: ['__id__', '__signature__']                             ]8;id=833576;file:///work/squeakily/squeakily/filter.py\filter.py]8;;\:]8;id=62270;file:///work/squeakily/squeakily/filter.py#330\330]8;;\

Indexing signatures...: 100%|██████████| 18014/18014 [00:01<00:00, 16967.57it/s]


[11/26/22 17:08:12] INFO     Dataset columns: ['__id__', '__neighbors__']                             ]8;id=434364;file:///work/squeakily/squeakily/filter.py\filter.py]8;;\:]8;id=640108;file:///work/squeakily/squeakily/filter.py#364\364]8;;\

Iterating over components...: 100%|██████████| 10560/10560 [00:00<00:00, 2443149.11it/s]


[11/26/22 17:08:25] INFO     Final dataset size: 10560                                             ]8;id=60711;file:///tmp/ipykernel_132178/1047258885.py\1047258885.py]8;;\:]8;id=116601;file:///tmp/ipykernel_132178/1047258885.py#25\25]8;;\

In [ ]:
logger.info(f"Final dataset size: {len(pipeline.datasources[0]['dataset'])}")

                    INFO     Final dataset size: 10560                                              ]8;id=844985;file:///tmp/ipykernel_132178/1381750264.py\1381750264.py]8;;\:]8;id=400237;file:///tmp/ipykernel_132178/1381750264.py#1\1]8;;\

In [ ]:
logger.info(f"Final dataset size: {len(pipeline.datasources[1]['dataset'])}")

                    INFO     Final dataset size: 18014                                               ]8;id=932506;file:///tmp/ipykernel_132178/382496783.py\382496783.py]8;;\:]8;id=82009;file:///tmp/ipykernel_132178/382496783.py#1\1]8;;\

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()